In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import impute
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
#-------------------------------------------------------------
prev_app = pd.read_csv('./data/previous_application.csv')
credit_card = pd.read_csv('./data/credit_card_balance.csv')
pos_cash = pd.read_csv('./data/POS_CASH_balance.csv')
inst_pmnt = pd.read_csv('./data/installments_payments.csv')

In [42]:
app_bur_train = pd.read_csv('./data/app_bur_train.csv')
app_bur_test = pd.read_csv('./data/app_bur_test.csv')

##### Previous Application

In [20]:
# aggregating certain numeric columns
prev_grp = prev_app.groupby('SK_ID_CURR', as_index = False).agg({'NAME_CONTRACT_TYPE': 'count', 'AMT_ANNUITY': ['mean', 'sum'],
                                                     'AMT_APPLICATION': ['mean', 'sum'], 'AMT_CREDIT': ['mean', 'sum'],
                                                      'AMT_DOWN_PAYMENT':['mean', 'max', 'sum'], 
                                                      'RATE_DOWN_PAYMENT': 'mean', 'RATE_INTEREST_PRIMARY': 'mean',
                                                      'RATE_INTEREST_PRIVILEGED': 'mean', 
                                                      'DAYS_LAST_DUE': ['mean', 'sum']
                                                     })

prev_grp.columns = ['SK_ID_CURR', 'cnt_NAME_CONTRACT_TYPE', 'mean_AMT_ANNUITY', 'sum_AMT_ANNUITY', 'mean_AMT_APPLICATION',
                   'sum_AMT_APPLICATION', 'mean_AMT_CREDIT', 'sum_AMT_CREDIT', 'mean_AMT_DOWN_PAYMENT',
                   'max_AMT_DOWN_PAYMENT', 'sum_AMT_DOWN_PAYMENT', 'mean_RATE_DOWN_PAYMENT',
                   'mean_RATE_INTEREST_PRIMARY', 'mean_RATE_INTEREST_PRIVILEGED', 'mean_DAYS_LAST_DUE',
                   'sum_DAYS_LAST_DUE']

# count of labels in a few categorical columns
dummy = pd.get_dummies(prev_app[['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'NAME_CONTRACT_STATUS']])
dummy = dummy.drop(['NAME_CONTRACT_TYPE_XNA', 'NAME_CONTRACT_STATUS_Unused offer'], axis = 1)
count = dummy.groupby('SK_ID_CURR', as_index = False).count()
count.columns = ['SK_ID_CURR', 'cnt_Cash loans', 'cnt_Consumer loans', 'cnt_Revolving loans', 'cnt_Approved', 
                 'cnt_Canceled', 'cnt_Refused']

# joining and standardizing
previous = pd.merge(prev_grp, count, on = 'SK_ID_CURR', how = 'outer')
previous.iloc[:, 1:] = previous.iloc[:, 1:].apply(lambda x: (x - np.mean(x)) / np.std(x), axis = 0)
previous = previous.fillna(0)

##### POS CASH

In [22]:
pos_cash['CNT_INSTALMENT_PAID'] = pos_cash['CNT_INSTALMENT'] - pos_cash['CNT_INSTALMENT_FUTURE']
pos_cash['ACTIVE_POS'] = pos_cash.NAME_CONTRACT_STATUS == 'Active'
pos_cash['POS_DPD'] = pos_cash.SK_DPD > 0
pos_group = pos_cash.groupby('SK_ID_CURR', as_index = False).agg({'MONTHS_BALANCE': 'sum', 'CNT_INSTALMENT_PAID': 'sum', 
                                    'CNT_INSTALMENT_FUTURE': 'sum', 'ACTIVE_POS': 'sum', 'POS_DPD': 'sum',
                                    'SK_DPD_DEF': ['sum', 'mean']})
pos_group.columns = ['SK_ID_CURR', 'tot_MONTHS_BALpos', 'cnt_INST_PAIDpos', 'cnt_INST_FUTUREpos', 
                     'cnt_ACTIVE_POS', 'cnt_POS_DPD', 'sum_DPD_DEFpos', 'mean_DPD_DEFpos']
pos_group.iloc[:, 1:] = pos_group.iloc[:, 1:].apply(lambda x: (x - np.mean(x)) / np.std(x), axis = 0)
pos_group = pos_group.fillna(0)
pos_group.head(2)

,SK_ID_CURR,tot_MONTHS_BALpos,cnt_INST_PAIDpos,cnt_INST_FUTUREpos,cnt_ACTIVE_POS,cnt_POS_DPD,sum_DPD_DEFpos,mean_DPD_DEFpos
0,100001,0.343334,-0.735632,-0.776749,-0.885926,0.025874,-0.008127,0.040747
1,100002,0.755891,-0.104076,-0.065603,-0.357918,-0.181766,-0.012712,-0.016634


#### Credit_Card_Balance

In [36]:
# new columns:
# whether there was a default on payment in last 3 months
credit_card['cc_def_3mth'] = credit_card.SK_DPD_DEF > 1
# how much extra amount was paid over minimum required amount
credit_card['cc_extra_paid'] = credit_card.AMT_PAYMENT_CURRENT - credit_card.AMT_INST_MIN_REGULARITY


# last 3 months
cc_3 = credit_card[credit_card.MONTHS_BALANCE >= -3]

# aggregation
cc3_gr = cc_3.groupby(['SK_ID_CURR', 'SK_ID_PREV'], as_index = False).mean()[['SK_ID_CURR', 'SK_ID_PREV',  
                                                   'AMT_BALANCE', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 
                                                   'cc_extra_paid', 'AMT_PAYMENT_CURRENT', 'AMT_TOTAL_RECEIVABLE', 
                                                   'CNT_DRAWINGS_ATM_CURRENT', 'SK_DPD', 'SK_DPD_DEF']]
cc3_gr = cc3_gr.groupby('SK_ID_CURR', as_index = False).sum()[['SK_ID_CURR', 'AMT_BALANCE', 'AMT_DRAWINGS_ATM_CURRENT', 
                                                               'AMT_DRAWINGS_CURRENT', 'cc_extra_paid', 
                                                               'AMT_PAYMENT_CURRENT', 'AMT_TOTAL_RECEIVABLE', 
                                                               'CNT_DRAWINGS_ATM_CURRENT', 'SK_DPD', 'SK_DPD_DEF']]
cc3_gr.columns = ['SK_ID_CURR', 'cc3_AMT_BALANCE', 'cc3_AMT_DRAWINGS_ATM_CURRENT', 'cc3_AMT_DRAWINGS_CURRENT', 
                  'cc3_extra_paid', 'cc3_AMT_PAYMENT_CURRENT', 'cc3_AMT_TOTAL_RECEIVABLE', 'cc3_CNT_DRAWINGS_ATM_CURRENT', 
                  'cc3_DPD', 'cc3_cnt_DPD_DEF']
cc3_gr.iloc[:, 1:] = cc3_gr.iloc[:, 1:].apply(lambda x: (x - np.mean(x)) / np.std(x), axis = 0)
cc3_gr = cc3_gr.fillna(0)


# last 2 years
cc_24 = credit_card[credit_card.MONTHS_BALANCE >= -24]

# aggregation
cc24_gr = cc_24.groupby(['SK_ID_CURR', 'SK_ID_PREV'], as_index = False).mean()[['SK_ID_CURR', 'MONTHS_BALANCE',
                                                    'AMT_BALANCE', 'SK_DPD_DEF']]
cc24_gr = cc_24.groupby('SK_ID_CURR', as_index = False).sum()[['SK_ID_CURR', 'MONTHS_BALANCE',
                                                    'AMT_BALANCE', 'SK_DPD_DEF']]
cc24_gr.columns = ['SK_ID_CURR', 'cc24_MONTHS_BALANCE', 'cc24_AMT_BALANCE', 'cc24_SK_DPD_DEF']
cc24_gr.iloc[:, 1:] = cc24_gr.iloc[:, 1:].apply(lambda x: (x - np.mean(x)) / np.std(x), axis = 0)
cc24_gr = cc24_gr.fillna(0)

ccbal_new = pd.merge(cc3_gr, cc24_gr, on = 'SK_ID_CURR', how = 'outer')
ccbal_new.head(2)

,SK_ID_CURR,cc3_AMT_BALANCE,cc3_AMT_DRAWINGS_ATM_CURRENT,cc3_AMT_DRAWINGS_CURRENT,cc3_extra_paid,cc3_AMT_PAYMENT_CURRENT,cc3_AMT_TOTAL_RECEIVABLE,cc3_CNT_DRAWINGS_ATM_CURRENT,cc3_DPD,cc3_cnt_DPD_DEF,cc24_MONTHS_BALANCE,cc24_AMT_BALANCE,cc24_SK_DPD_DEF
0,100006,-0.518308,-0.224959,-0.28656,-0.199098,-0.308883,-0.517673,-0.282435,-0.073671,-0.010731,1.430216,-0.509711,-0.011234
1,100011,-0.518308,-0.224959,-0.28656,-0.179582,-0.290622,-0.517673,-0.282435,-0.073671,-0.010731,-0.921048,-0.509711,-0.011234


#### installments_payments

In [39]:
# whether late and how much under/over-paid
inst_pmnt['late'] = inst_pmnt['DAYS_INSTALMENT'] - inst_pmnt['DAYS_ENTRY_PAYMENT'] < 0
inst_pmnt['underpaid'] = inst_pmnt['AMT_INSTALMENT'] - inst_pmnt['AMT_PAYMENT']

# for last 1-yr and last 3-yrs
inst_1yr = inst_pmnt[inst_pmnt.DAYS_INSTALMENT > -365]
inst_3yr = inst_pmnt[inst_pmnt.DAYS_INSTALMENT > -1095]

# aggregation
inst_1yr_gr = inst_1yr.groupby('SK_ID_CURR', as_index = False).sum()[['SK_ID_CURR', 'late', 'underpaid']]
inst_1yr_gr.columns = ['SK_ID_CURR', 'inst1y_cnt_late', 'inst1y_val_underpaid']

inst_3yr_gr = inst_3yr.groupby('SK_ID_CURR', as_index = False).sum()[['SK_ID_CURR', 'late', 'underpaid']]
inst_3yr_gr.columns = ['SK_ID_CURR', 'inst3y_cnt_late', 'ins3y_val_underpaid']

# standardizing
inst_1yr_gr.iloc[:, 1:] = inst_1yr_gr.iloc[:, 1:].apply(lambda x: (x - np.mean(x)) / np.std(x), axis = 0)
inst_1yr_gr = inst_1yr_gr.fillna(0)

inst_3yr_gr.iloc[:, 1:] = inst_3yr_gr.iloc[:, 1:].apply(lambda x: (x - np.mean(x)) / np.std(x), axis = 0)
inst_3yr_gr = inst_3yr_gr.fillna(0)

# merged
inst_new = pd.merge(inst_1yr_gr, inst_3yr_gr, on = 'SK_ID_CURR', how = 'outer')
inst_new.head(2)

,SK_ID_CURR,inst1y_cnt_late,inst1y_val_underpaid,inst3y_cnt_late,ins3y_val_underpaid
0,100002,-0.414405,0.107771,-0.452169,0.098616
1,100006,-0.414405,0.107771,-0.452169,0.098616


In [41]:
previous1 = pd.merge(previous, pos_group, on = 'SK_ID_CURR', how = 'outer')
previous2 = pd.merge(previous1, ccbal_new, on = 'SK_ID_CURR', how = 'outer')
previous3 = pd.merge(previous2, inst_new, on = 'SK_ID_CURR', how = 'outer')

In [43]:
merged_train = pd.merge(app_bur_train, previous3, on = 'SK_ID_CURR', how = 'left')
merged_train = merged_train.fillna(0)
merged_test = pd.merge(app_bur_test, previous3, on = 'SK_ID_CURR', how = 'left')
merged_test = merged_test.fillna(0)

In [48]:
# merged_train.to_csv('./data/merged_train.csv', index = False)
# merged_test.to_csv('./data/merged_test.csv', index = False)